In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import os
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
dataset = pd.read_csv("./spam.csv")

In [4]:
dataset.head()

,text,target
0,From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...,0
1,From gort44@excite.com Mon Jun 24 17:54:21 200...,1
2,From fork-admin@xent.com Mon Jul 29 11:39:57 2...,1
3,From dcm123@btamail.net.cn Mon Jun 24 17:49:23...,1
4,From ilug-admin@linux.ie Mon Aug 19 11:02:47 2...,0


In [4]:
data, target = dataset.text, dataset.target

In [5]:
data.shape

(5796,)

In [6]:
target.shape

(5796,)

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5796 entries, 0 to 5795
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5796 non-null   object
 1   target  5796 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 90.7+ KB


In [8]:
import matplotlib as mpl 
import matplotlib.pyplot as plt

In [9]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for train_index, test_index in sss.split(data, target):
    train_X, test_X = data.loc[train_index], data.loc[test_index]
    train_y, test_y = target.loc[train_index], target.loc[test_index]
    


In [10]:
from sklearn.model_selection import cross_val_score, cross_val_predict



In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5796 entries, 0 to 5795
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5796 non-null   object
 1   target  5796 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 90.7+ KB


In [13]:
vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(train_X)
X_train_vectorized = vect.transform(train_X)


In [14]:
def add_feature(X, feature_to_add):
   
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')
 
 

add_length=train_X.str.len()
add_digits=train_X.str.count(r'\d')
add_dollars=train_X.str.count(r'\$')
add_characters=train_X.str.count(r'\W')
 
X_train_transformed = add_feature(X_train_vectorized , [add_length, add_digits,  add_dollars, add_characters])
 

add_length_t=test_X.str.len()
add_digits_t=test_X.str.count(r'\d')
add_dollars_t=test_X.str.count(r'\$')
add_characters_t=test_X.str.count(r'\W')
 
 
X_test_transformed = add_feature(vect.transform(test_X), [add_length_t, add_digits_t,  add_dollars_t, add_characters_t])

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
forest_clf = RandomForestClassifier(random_state = 42)
y_probas_forest = cross_val_predict(forest_clf , X_train_transformed , train_y , cv = 3 , method = 'predict_proba')

In [17]:
from sklearn.model_selection import RandomizedSearchCV

In [18]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = [
    {'n_estimators' : [3, 10, 30], 'max_features' : [2, 4, 6, 8]},
    {'bootstrap' : [False], 'n_estimators' : [3, 10], 'max_features' : [2, 3, 4]},
    
]

forest_reg = RandomForestClassifier()

random_search = RandomizedSearchCV(forest_reg, param_grid, cv = 5,
                          scoring = 'neg_mean_squared_error',
                          return_train_score = True)

random_search.fit(X_train_transformed , train_y)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions=[{'max_features': [2, 4, 6, 8],
                                         'n_estimators': [3, 10, 30]},
                                        {'bootstrap': [False],
                                         'max_features': [2, 3, 4],
                                         'n_estimators': [3, 10]}],
                   return_train_score=True, scoring='neg_mean_squared_error')

In [19]:
random_search.best_params_

{'n_estimators': 10, 'max_features': 4, 'bootstrap': False}

In [20]:
random_search.best_estimator_

RandomForestClassifier(bootstrap=False, max_features=4, n_estimators=10)

In [21]:
final_model = random_search.best_estimator_

In [22]:
final_model.fit(X_train_transformed, train_y)

RandomForestClassifier(bootstrap=False, max_features=4, n_estimators=10)

In [23]:
predictions = final_model.predict(X_test_transformed)

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
accuracy_score(predictions, test_y)

0.9862068965517241